In [4]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [15]:
df =  pd.read_csv(r'E:/ANZ JO/Mini Project/Sarcasm Analyser/toxicity.csv')


In [16]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [10]:
from tensorflow import keras
from keras.layers import TextVectorization

In [17]:
X = df['comment_text']
y = df[df.columns[2:]].values
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [12]:
MAX_FEATURES = 200000

In [13]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [18]:
vectorizer.adapt(X.values)

In [19]:
vectorized_text = vectorizer(X.values)

In [10]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [11]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [13]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [14]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [15]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 6340s 908ms/step - loss: 0.0614 - val_loss: 0.0460


In [16]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [18]:
res = model.predict(np.expand_dims(input_text,0))
res

1/1 [==============================] - 1s 554ms/step


array([[0.9936385 , 0.18146862, 0.9456314 , 0.02153116, 0.78668064,
        0.17589052]], dtype=float32)

In [19]:
model.save('toxic_model')

INFO:tensorflow:Assets written to: toxic_model\assets


INFO:tensorflow:Assets written to: toxic_model\assets


In [21]:
print(df.columns[2:])

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


In [6]:
model = tf.keras.models.load_model('toxic_model')

In [20]:
input_str = vectorizer('hey i freaken hate you!')
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 758ms/step


In [7]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [22]:
score_comment('I kill you')

1/1 [==============================] - 0s 146ms/step


'toxic: True\nsevere_toxic: False\nobscene: True\nthreat: False\ninsult: False\nidentity_hate: False\n'